In [15]:
import os
from datetime import datetime

import pandas as pd
import requests
from bs4 import BeautifulSoup
from dotenv import find_dotenv
from dotenv import load_dotenv

In [16]:
def get_soup(url):
    try:
        HEADERS = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }
        response = requests.get(url, headers=HEADERS)
        return BeautifulSoup(response.content, "html.parser")
    except Exception as e:
        print(f"Failed to get soup from url {url}. Error: {e}")
        raise

In [17]:
crop = "soja"
URL = f"https://www.agrolink.com.br/noticias/cultura/{crop}/lista"
soup = get_soup(URL)

In [18]:
news_divs = soup.find_all('div', class_='col-auto col-blocks')

In [25]:
news_list = []

for div in news_divs:
    news_item = {}
    
    # Extrair o título da notícia
    news_title_tag = div.find('a', class_='news-list-main-link-title')
    if news_title_tag:
        news_item['news_title'] = news_title_tag.h4.text.strip()
        news_item['link'] = news_title_tag['href']
        news_item['link'] = "https://www.agrolink.com.br" + news_item['link']
    
    # Extrair a data
    date_tag = div.find_all('a')[1]
    if date_tag:
        news_item['date_news'] = date_tag.text.strip().split(' ')[-2]
    
    # Extrair a descrição da notícia
    news_sub_tag = div.find('a', class_='news-list-main-link-description')
    if news_sub_tag:
        news_item['news_sub'] = news_sub_tag.text.strip()
    
    if news_item:
        news_list.append(news_item)

In [30]:
df = pd.DataFrame(news_list)

In [26]:
def create_dataframe(data_list, crop):
    try:
        df = pd.concat(
            [
                pd.DataFrame(
                    {"date_news": data["date_news"], "news_title": data["news_title"], "news_sub": data["news_sub"], "link": data["link"], "crop": crop}
                )
                for data in data_list
            ],
            ignore_index=True,
        )
        df["date_news"] = pd.to_datetime(
            df["date_news"], errors="coerce", dayfirst=True
        )
        return df
    except Exception as e:
        print(f"Failed to create dataframe. Error: {e}")
        raise

In [31]:
df.head()

,news_title,link,date_news,news_sub
0,Queda geral para os preços da soja,https://www.agrolink.com.br/noticias/cultura/s...,31/07/2024,As quedas expressivas também ocorreram no Mato...
1,Soja negociada em Chicago fecha em baixa,https://www.agrolink.com.br/noticias/cultura/s...,31/07/2024,As cotações da oleaginosa seguem em queda acen...
2,Avanço na regulação torna Brasil promissor em ...,https://www.agrolink.com.br/noticias/cultura/s...,31/07/2024,“O crescimento dos bioinsumos no Brasil enfren...
3,Saiba o que está influenciando os preços da soja,https://www.agrolink.com.br/noticias/cultura/s...,31/07/2024,Mercado da soja está atravessando um período d...
4,Queda na produção nacional impulsiona importaç...,https://www.agrolink.com.br/noticias/cultura/s...,30/07/2024,Paraguai é o principal fornecedor


In [33]:
df['date_news'] = pd.to_datetime(df['date_news'], errors='coerce', dayfirst=True)

In [35]:
df.head()

,news_title,link,date_news,news_sub
0,Queda geral para os preços da soja,https://www.agrolink.com.br/noticias/cultura/s...,2024-07-31,As quedas expressivas também ocorreram no Mato...
1,Soja negociada em Chicago fecha em baixa,https://www.agrolink.com.br/noticias/cultura/s...,2024-07-31,As cotações da oleaginosa seguem em queda acen...
2,Avanço na regulação torna Brasil promissor em ...,https://www.agrolink.com.br/noticias/cultura/s...,2024-07-31,“O crescimento dos bioinsumos no Brasil enfren...
3,Saiba o que está influenciando os preços da soja,https://www.agrolink.com.br/noticias/cultura/s...,2024-07-31,Mercado da soja está atravessando um período d...
4,Queda na produção nacional impulsiona importaç...,https://www.agrolink.com.br/noticias/cultura/s...,2024-07-30,Paraguai é o principal fornecedor


In [36]:
def filter_by_today(df: pd.DataFrame):
    try:
        today = datetime.today().strftime("%Y-%m-%d")
        return df[df["date_news"] == today]
    except Exception as e:
        print(f"Failed to filter dataframe by today. Error: {e}")
        raise

In [37]:
df = filter_by_today(df)
print(f"Number of news: {len(df)}")

Number of news: 4


In [38]:
df

,news_title,link,date_news,news_sub
0,Queda geral para os preços da soja,https://www.agrolink.com.br/noticias/cultura/s...,2024-07-31,As quedas expressivas também ocorreram no Mato...
1,Soja negociada em Chicago fecha em baixa,https://www.agrolink.com.br/noticias/cultura/s...,2024-07-31,As cotações da oleaginosa seguem em queda acen...
2,Avanço na regulação torna Brasil promissor em ...,https://www.agrolink.com.br/noticias/cultura/s...,2024-07-31,“O crescimento dos bioinsumos no Brasil enfren...
3,Saiba o que está influenciando os preços da soja,https://www.agrolink.com.br/noticias/cultura/s...,2024-07-31,Mercado da soja está atravessando um período d...
